In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
raw_description = pd.read_csv(
    "raw_description.csv"
)

In [3]:
raw_description

,periodo,tipo_exam,descripcion,link
0,ADMISIÓN 2019-I,CEPUNT,Resultados Cepunt Trujillo,http://admisionunt.info/docs/padrones/20191/20...
1,ADMISIÓN 2019-I,CEPUNT,Resultados Cepunt Valle Jequetepeque,http://admisionunt.info/docs/padrones/20191/20...
2,ADMISIÓN 2019-I,CEPUNT,Resultados Cepunt Huamachuco,http://admisionunt.info/docs/padrones/20191/20...
3,ADMISIÓN 2019-I,ORDINARIO,Resultados Ordinario Área A Trujillo,http://admisionunt.info/docs/padrones/20191/20...
4,ADMISIÓN 2019-I,ORDINARIO,Resultados Ordinario Área B Trujillo,http://admisionunt.info/docs/padrones/20191/20...
...,...,...,...,...
207,ADMISION 2024-II,ORDINARIO,Áreas A y D: Huamachuco,https://unitru.edu.pe/webfiles///Convocatoria/...
208,ADMISION 2024-II,ORDINARIO,Áreas B y C: Huamachuco,https://unitru.edu.pe/webfiles///Convocatoria/...
209,ADMISION 2024-II,ORDINARIO,Áreas A y D: Valle Jequetepeque,https://unitru.edu.pe/webfiles///Convocatoria/...
210,ADMISION 2024-II,ORDINARIO,Áreas B y C: Valle Jequetepeque,https://unitru.edu.pe/webfiles///Convocatoria/...


# DELETE SUMATIVOS I

In [4]:
sumativos_filter = (
    raw_description
    ['descripcion']
    .str.contains("Sumativo")
)
sumativos_filter

0      False
1      False
2      False
3      False
4      False
       ...  
207    False
208    False
209    False
210    False
211    False
Name: descripcion, Length: 212, dtype: bool

In [5]:
sumativos = (
    raw_description
    ['descripcion']
    .loc[sumativos_filter]
)
sumativos

178               Resultados I Sumativo CEPUNT: Trujillo
179              Resultados II Sumativo CEPUNT: Trujillo
180     Resultados I Sumativo CEPUNT: Valle Jequetepeque
181    Resultados II Sumativo CEPUNT: Valle Jequetepeque
182             Resultados I Sumativo CEPUNT: Huamachuco
183            Resultados II Sumativo CEPUNT: Huamachuco
192                        I Sumativo CEPUNT: Huamachuco
193                I Sumativo CEPUNT: Valle Jequetepeque
194                          I Sumativo CEPUNT: Trujillo
195                       II Sumativo CEPUNT: Huamachuco
196               II Sumativo CEPUNT: Valle Jequetepeque
197                         II Sumativo CEPUNT: Trujillo
Name: descripcion, dtype: object

In [6]:
sumativos_ii_filter = (
    sumativos
    .str.contains("II Sumativo")
)
sumativos_ii_filter

178    False
179     True
180    False
181     True
182    False
183     True
192    False
193    False
194    False
195     True
196     True
197     True
Name: descripcion, dtype: bool

In [7]:
sumativos_i_index = (
    sumativos
    .loc[~sumativos_ii_filter]
    .index
)

sumativos_i_index

Index([178, 180, 182, 192, 193, 194], dtype='int64')

In [8]:
raw_description = (
    raw_description
    .drop(sumativos_i_index)
    .reset_index()
    .drop(['index'], axis=1)
)

raw_description

,periodo,tipo_exam,descripcion,link
0,ADMISIÓN 2019-I,CEPUNT,Resultados Cepunt Trujillo,http://admisionunt.info/docs/padrones/20191/20...
1,ADMISIÓN 2019-I,CEPUNT,Resultados Cepunt Valle Jequetepeque,http://admisionunt.info/docs/padrones/20191/20...
2,ADMISIÓN 2019-I,CEPUNT,Resultados Cepunt Huamachuco,http://admisionunt.info/docs/padrones/20191/20...
3,ADMISIÓN 2019-I,ORDINARIO,Resultados Ordinario Área A Trujillo,http://admisionunt.info/docs/padrones/20191/20...
4,ADMISIÓN 2019-I,ORDINARIO,Resultados Ordinario Área B Trujillo,http://admisionunt.info/docs/padrones/20191/20...
...,...,...,...,...
201,ADMISION 2024-II,ORDINARIO,Áreas A y D: Huamachuco,https://unitru.edu.pe/webfiles///Convocatoria/...
202,ADMISION 2024-II,ORDINARIO,Áreas B y C: Huamachuco,https://unitru.edu.pe/webfiles///Convocatoria/...
203,ADMISION 2024-II,ORDINARIO,Áreas A y D: Valle Jequetepeque,https://unitru.edu.pe/webfiles///Convocatoria/...
204,ADMISION 2024-II,ORDINARIO,Áreas B y C: Valle Jequetepeque,https://unitru.edu.pe/webfiles///Convocatoria/...


# CLEAN COLUMN PERIODO

In [9]:
raw_description['periodo'].value_counts()

periodo
ADMISION 2022-II    25
ADMISION 2023-II    25
ADMISIÓN 2021-II    24
ADMISIÓN 2021-I     18
ADMISIÓN 2019-II    17
ADMISIÓN 2020-II    17
ADMISION 2022-I     17
ADMISION 2023-I     17
ADMISION 2024-II    17
ADMISION 2024-I     11
ADMISIÓN 2019-I      9
ADMISIÓN 2020-I      9
Name: count, dtype: int64

In [10]:
periods = (
    raw_description
    ['periodo']
    .str.split(" ", expand=True)
    .drop(0, axis=1)
    [1]
    .str.split("-", expand=True)
)
periods.columns = ['fecha_periodo', 'numero_periodo']
periods

,fecha_periodo,numero_periodo
0,2019,I
1,2019,I
2,2019,I
3,2019,I
4,2019,I
...,...,...
201,2024,II
202,2024,II
203,2024,II
204,2024,II


In [15]:
periods['fecha_periodo'] = periods['fecha_periodo'].astype('int64')

In [16]:
periods['fecha_periodo']

0      2019
1      2019
2      2019
3      2019
4      2019
       ... 
201    2024
202    2024
203    2024
204    2024
205    2024
Name: fecha_periodo, Length: 206, dtype: int64

In [18]:
periods['fecha_periodo']

0      2019
1      2019
2      2019
3      2019
4      2019
       ... 
201    2024
202    2024
203    2024
204    2024
205    2024
Name: fecha_periodo, Length: 206, dtype: int64

# CLEAN COLUMN DESCRIPCION

In [12]:
raw_description['descripcion'].value_counts()

descripcion
Resultados Ordinario Área A Trujillo                                 11
Resultados Ordinario Área B Trujillo                                 10
Resultados Excelencia Trujillo                                       10
Resultados Ordinario 5to Secundaria Área A Trujillo                   6
Resultados Personas con Discapacidad Trujillo                         6
                                                                     ..
Resultados Persona con Discapacidad                                   1
Resultados Victimas de la Violencia Excelencia Valle Jequetepeque     1
Resultados Ordinario Área A Valle Jequetepeque                        1
Resultados Ordinario Área B Valle Jequetepeque                        1
Áreas B y C: Santiago de Chuco                                        1
Name: count, Length: 78, dtype: int64

In [ ]:
# only for description
(
    raw_description
    ['descripcion']
    .value_counts()
    .index
    .sort_values()
)

## REPLACE VALUES

In [16]:
many_white_spaces = re.compile(r'\s+')
area = re.compile(r'Áreas?-?')

description_values = (
    raw_description['descripcion']
    .str.replace("Cepunt", "", regex=False)
    .str.replace("CEPUNT", "", regex=False)
    .str.replace("Ordinario", "", regex=False)
    .str.replace("Resultados", "", regex=False)
    .str.replace("2024-II", "", regex=False)
    .str.replace("II Sumativo", "", regex=False)
    .str.replace(":", "", regex=False)
    .str.replace("-", "", regex=False)
    .str.replace(" y ", "-", regex=False)
    .str.replace(" Y ", "-", regex=False)
    .str.replace(area, "", regex=True)
    .str.replace(many_white_spaces, " ", regex=True)
    .str.strip()
    # .value_counts()
    # .sort_index()
    # .head(40)
)

description_values

0                    Trujillo
1          Valle Jequetepeque
2                  Huamachuco
3                  A Trujillo
4                  B Trujillo
                ...          
201            A-D Huamachuco
202            B-C Huamachuco
203    A-D Valle Jequetepeque
204    B-C Valle Jequetepeque
205     B-C Santiago de Chuco
Name: descripcion, Length: 206, dtype: object

## COLUMN PLACE

In [17]:
places = re.compile(r'(Trujillo|Huamachuco|Sa.tiago\sde\sChuco|Valle\sJequetepeque|Valle)')

column_place = (
    description_values
    .str.extract(places)
    [0]
    .str.replace("Samtiago", "Santiago", regex=False)
)

column_place.name = "lugar"
column_place

0                Trujillo
1      Valle Jequetepeque
2              Huamachuco
3                Trujillo
4                Trujillo
              ...        
201            Huamachuco
202            Huamachuco
203    Valle Jequetepeque
204    Valle Jequetepeque
205     Santiago de Chuco
Name: lugar, Length: 206, dtype: object

## COLUMN MOD

In [18]:
mods = re.compile(
    r'(5to\sSecundaria|Deportistas\sCalificados|Personas?\scon\sDiscapacidad|V.ctimas\sde\sla\sViolencia\sExcelencia|V.ctimas\sde\sla\sViolencia|Excelencia)'
    )

column_mod = (
    description_values
    .str.extract(mods)
    [0]
    .str.replace("Víctimas", "Victimas", regex=False)
    .str.replace("Personas", "Persona", regex=False)
    .str.replace("Persona", "Personas", regex=False)
    # .value_counts()
)

column_mod.name = "modalidad"
column_mod

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
201    NaN
202    NaN
203    NaN
204    NaN
205    NaN
Name: modalidad, Length: 206, dtype: object

# JOIN COLUMNS

In [19]:
joined_columns = pd.concat(
    [
        raw_description,
        periods,
        column_mod,
        column_place
    ],
    axis=1
)

joined_columns

,periodo,tipo_exam,descripcion,link,fecha_periodo,numero_periodo,modalidad,lugar
0,ADMISIÓN 2019-I,CEPUNT,Resultados Cepunt Trujillo,http://admisionunt.info/docs/padrones/20191/20...,2019,I,NaN,Trujillo
1,ADMISIÓN 2019-I,CEPUNT,Resultados Cepunt Valle Jequetepeque,http://admisionunt.info/docs/padrones/20191/20...,2019,I,NaN,Valle Jequetepeque
2,ADMISIÓN 2019-I,CEPUNT,Resultados Cepunt Huamachuco,http://admisionunt.info/docs/padrones/20191/20...,2019,I,NaN,Huamachuco
3,ADMISIÓN 2019-I,ORDINARIO,Resultados Ordinario Área A Trujillo,http://admisionunt.info/docs/padrones/20191/20...,2019,I,NaN,Trujillo
4,ADMISIÓN 2019-I,ORDINARIO,Resultados Ordinario Área B Trujillo,http://admisionunt.info/docs/padrones/20191/20...,2019,I,NaN,Trujillo
...,...,...,...,...,...,...,...,...
201,ADMISION 2024-II,ORDINARIO,Áreas A y D: Huamachuco,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,NaN,Huamachuco
202,ADMISION 2024-II,ORDINARIO,Áreas B y C: Huamachuco,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,NaN,Huamachuco
203,ADMISION 2024-II,ORDINARIO,Áreas A y D: Valle Jequetepeque,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,NaN,Valle Jequetepeque
204,ADMISION 2024-II,ORDINARIO,Áreas B y C: Valle Jequetepeque,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,NaN,Valle Jequetepeque


In [20]:
deleted_columns = (
    joined_columns
    .drop(['periodo', 'descripcion'], axis=1)
    .reindex(columns=['link', 'fecha_periodo', 'numero_periodo', 'tipo_exam', 'modalidad', 'lugar'])
)
deleted_columns

,link,fecha_periodo,numero_periodo,tipo_exam,modalidad,lugar
0,http://admisionunt.info/docs/padrones/20191/20...,2019,I,CEPUNT,NaN,Trujillo
1,http://admisionunt.info/docs/padrones/20191/20...,2019,I,CEPUNT,NaN,Valle Jequetepeque
2,http://admisionunt.info/docs/padrones/20191/20...,2019,I,CEPUNT,NaN,Huamachuco
3,http://admisionunt.info/docs/padrones/20191/20...,2019,I,ORDINARIO,NaN,Trujillo
4,http://admisionunt.info/docs/padrones/20191/20...,2019,I,ORDINARIO,NaN,Trujillo
...,...,...,...,...,...,...
201,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ORDINARIO,NaN,Huamachuco
202,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ORDINARIO,NaN,Huamachuco
203,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ORDINARIO,NaN,Valle Jequetepeque
204,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ORDINARIO,NaN,Valle Jequetepeque


# FILLING SOME MISSING DATA

In [22]:
# deleted_columns.loc[(
#     deleted_columns
#     ['tipo_exam'][deleted_columns['tipo_exam'] == "EXTRAORDINARIO"]
#     .index
# )]['modalidad'].value_counts().sum()

In [23]:

# deleted_columns['tipo_exam'].value_counts()

In [21]:
filter_extraor_lugar_nan = (
    (deleted_columns['tipo_exam'] == "EXTRAORDINARIO") & (deleted_columns['lugar'].isna())
)

deleted_columns.loc[filter_extraor_lugar_nan, 'lugar'] = "Trujillo"

deleted_columns

,link,fecha_periodo,numero_periodo,tipo_exam,modalidad,lugar
0,http://admisionunt.info/docs/padrones/20191/20...,2019,I,CEPUNT,NaN,Trujillo
1,http://admisionunt.info/docs/padrones/20191/20...,2019,I,CEPUNT,NaN,Valle Jequetepeque
2,http://admisionunt.info/docs/padrones/20191/20...,2019,I,CEPUNT,NaN,Huamachuco
3,http://admisionunt.info/docs/padrones/20191/20...,2019,I,ORDINARIO,NaN,Trujillo
4,http://admisionunt.info/docs/padrones/20191/20...,2019,I,ORDINARIO,NaN,Trujillo
...,...,...,...,...,...,...
201,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ORDINARIO,NaN,Huamachuco
202,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ORDINARIO,NaN,Huamachuco
203,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ORDINARIO,NaN,Valle Jequetepeque
204,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ORDINARIO,NaN,Valle Jequetepeque


In [22]:
filter_extraor_mod_nan = (
    (deleted_columns['tipo_exam'] == "EXTRAORDINARIO") & (deleted_columns['modalidad'].isna())
)

deleted_columns.loc[filter_extraor_mod_nan, 'modalidad'] = "Excelencia"

deleted_columns

,link,fecha_periodo,numero_periodo,tipo_exam,modalidad,lugar
0,http://admisionunt.info/docs/padrones/20191/20...,2019,I,CEPUNT,NaN,Trujillo
1,http://admisionunt.info/docs/padrones/20191/20...,2019,I,CEPUNT,NaN,Valle Jequetepeque
2,http://admisionunt.info/docs/padrones/20191/20...,2019,I,CEPUNT,NaN,Huamachuco
3,http://admisionunt.info/docs/padrones/20191/20...,2019,I,ORDINARIO,NaN,Trujillo
4,http://admisionunt.info/docs/padrones/20191/20...,2019,I,ORDINARIO,NaN,Trujillo
...,...,...,...,...,...,...
201,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ORDINARIO,NaN,Huamachuco
202,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ORDINARIO,NaN,Huamachuco
203,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ORDINARIO,NaN,Valle Jequetepeque
204,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ORDINARIO,NaN,Valle Jequetepeque


In [23]:
filter_ordi_mod_nan = (
    (deleted_columns['tipo_exam'] == "ORDINARIO") & (deleted_columns['modalidad'].isna())
)
deleted_columns.loc[filter_ordi_mod_nan, 'modalidad'] = "Normal"
deleted_columns

,link,fecha_periodo,numero_periodo,tipo_exam,modalidad,lugar
0,http://admisionunt.info/docs/padrones/20191/20...,2019,I,CEPUNT,NaN,Trujillo
1,http://admisionunt.info/docs/padrones/20191/20...,2019,I,CEPUNT,NaN,Valle Jequetepeque
2,http://admisionunt.info/docs/padrones/20191/20...,2019,I,CEPUNT,NaN,Huamachuco
3,http://admisionunt.info/docs/padrones/20191/20...,2019,I,ORDINARIO,Normal,Trujillo
4,http://admisionunt.info/docs/padrones/20191/20...,2019,I,ORDINARIO,Normal,Trujillo
...,...,...,...,...,...,...
201,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ORDINARIO,Normal,Huamachuco
202,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ORDINARIO,Normal,Huamachuco
203,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ORDINARIO,Normal,Valle Jequetepeque
204,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ORDINARIO,Normal,Valle Jequetepeque


In [24]:
filter_cepu_mod_nan = (
    (deleted_columns['tipo_exam'] == "CEPUNT") & (deleted_columns['modalidad'].isna())
)
deleted_columns.loc[filter_cepu_mod_nan, 'modalidad'] = "Normal"
deleted_columns

,link,fecha_periodo,numero_periodo,tipo_exam,modalidad,lugar
0,http://admisionunt.info/docs/padrones/20191/20...,2019,I,CEPUNT,Normal,Trujillo
1,http://admisionunt.info/docs/padrones/20191/20...,2019,I,CEPUNT,Normal,Valle Jequetepeque
2,http://admisionunt.info/docs/padrones/20191/20...,2019,I,CEPUNT,Normal,Huamachuco
3,http://admisionunt.info/docs/padrones/20191/20...,2019,I,ORDINARIO,Normal,Trujillo
4,http://admisionunt.info/docs/padrones/20191/20...,2019,I,ORDINARIO,Normal,Trujillo
...,...,...,...,...,...,...
201,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ORDINARIO,Normal,Huamachuco
202,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ORDINARIO,Normal,Huamachuco
203,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ORDINARIO,Normal,Valle Jequetepeque
204,https://unitru.edu.pe/webfiles///Convocatoria/...,2024,II,ORDINARIO,Normal,Valle Jequetepeque
